In [22]:
# %pip install nbformat

# %pip install requests

# %pip install pandas
# %pip install numpy

# %pip install keras
# %pip install tensorflow

# %pip install plotly

> 1. Изучить документацию RESTful API Binance (https://binance-docs.github.io/apidocs/spot/en/) и написать скрипт на Python с использованием библиотеки Requests, чтобы получить данные о котировках криптовалют. Сохранить полученные данные в формате CSV.

In [23]:
# импортирование необходимых библиотек
import os
import re
import math
import csv
import requests
import datetime


Что бы не продить множество лишних переменных, создам конструкт с настройками:

In [24]:
class Set:
    ''' 
    Основные настройки
    '''
    def __init__(self):
        pass

    url = 'https://api.binance.com/api/v3/klines'  # Доступ к данным

    headers = [
        'open_datetime', 'open', 'high', 'low', 'close', 'volume',
        'close_datetime', 'quote_asset_volume', 'number_of_trades',
        'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'
    ]

    ''' 
    Устанавливаем параметры: временные рамки, 
    время начала и окончания.
    Запускаем функцию получения и сохранения данных
    '''
    # Начальное и конечное время для получения данных
    start_time = "2023-04-01 00:00:00"  # формат: "год-месяц-день час:минута:секунда"
    end_time = "2023-05-01 00:00:00"  # формат: "год-месяц-день час:минута:секунда"
    
    # Временной интервал и лимит для получения данных
    timeframe = "5m"  # формат: "число + буква, например "1h" для получения данных за 1 час

    # Начальное и конечное время в миллисекундах
    start_time_ms = int(datetime.datetime.strptime(start_time, "%Y-%m-%d %H:%M:%S").timestamp() * 1000)
    end_time_ms = int(datetime.datetime.strptime(end_time, "%Y-%m-%d %H:%M:%S").timestamp() * 1000)

    # Котировка и путь
    symbol = 'BTC'+'USDT'
    path = 'data/'


Признаться, есть более лаконичный и передовой способ сбора информации с бирж, это CCXT. Но в рамках задания было предложенно использовать библиотеку requests, так что так и поступим. Возможно в будущем обновлю этот ноутбук.

In [25]:
# Подготовлю функции что бы не нагружать ноутбук множеством строк. В будущем я бы отправил их в отдельный конструкт или даже можно вынести в отдельный py файл все необходимые функции, но сейчас пусть они побудут тут для наглядности

def parse_timeframe(timeframe):
    '''
    Определяет тайм фрейм и разбивает на составляющие, количество и размер (минуты, часы, дни) этого достаточно, но можно будет добавить и другие при необходимости
    '''
    time = int(re.findall("[0-9]+", timeframe)[0])
    frame = str(re.findall("[a-zA-Z]+", timeframe)[0])
    if frame == 'm':
        divider = 60 * 1000
    elif frame == 'h':
        divider = 60 * 60 * 1000
    elif frame == 'd':    
        divider = 60 * 60 * 24 * 1000
    return time, divider

# 
def calculate_delta_and_iterations(start_time_ms, end_time_ms, timeframe):
    '''
    функция для расчета дельты и итераций для определения промежутка времени в миллисекундах
    '''
    time, divider = parse_timeframe(timeframe)
    delta = int((end_time_ms - start_time_ms) / divider / time)
    print(delta)
    limit = 1000
    rest = delta % limit
    # print(rest)
    if delta < limit:
        iterations = []
    else:
        iterations = [(limit, math.floor(delta / limit))]
    if rest > 0:
        iterations.append((rest, 1))
    # print(iterations, time, divider)
    return iterations, time, divider

def fetch_data(symbol, timeframe, start_time_ms, end_time_ms, path='data/'):
    '''
    Функция обходит лимит в 1000 баров повторяя запрос нужное количество раз
    '''
    if not os.path.exists(path):
        os.makedirs(path)

    iterations, time, divider = calculate_delta_and_iterations(
        start_time_ms, end_time_ms, timeframe)

    whole_data = []
    timestamp = start_time_ms

    for (pack, iterations_count) in iterations:
        for _ in range(iterations_count):
            params = {
                'symbol': symbol,
                'interval': timeframe,
                'startTime': timestamp,
                'limit': pack
            }
            response = requests.get(Set.url, params=params)
            whole_data.extend(response.json())

            timestamp += (pack * divider * time)

    with open(f'{path}/{symbol.lower()}_{timeframe}_data.csv', 'w', newline='') as f:
        writer = csv.writer(f)
        writer.writerow(Set.headers[:-1])
        for row in whole_data:
            writer.writerow(row[:len(Set.headers)-1])
    
    return iterations


Добываем данные

In [26]:
fetch_data(Set.symbol, Set.timeframe, Set.start_time_ms, Set.end_time_ms)


8640


[(1000, 8), (640, 1)]

> 2. Загрузить полученный CSV-файл с данными в Python с использованием библиотеки Pandas и выполнить предварительную обработку данных (например, удаление пропусков, преобразование даты и времени, масштабирование).

In [27]:
import pandas as pd
import numpy as np


import keras
from keras.preprocessing.sequence import TimeseriesGenerator
import tensorflow as tf

import plotly.graph_objects as go


In [28]:
filename = f'{Set.path}{Set.symbol.lower()}_{Set.timeframe}_data.csv'
df = pd.read_csv(filename)
print(df.info())


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3978 entries, 0 to 3977
Data columns (total 11 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   open_datetime                 3978 non-null   int64  
 1   open                          3978 non-null   float64
 2   high                          3978 non-null   float64
 3   low                           3978 non-null   float64
 4   close                         3978 non-null   float64
 5   volume                        3978 non-null   float64
 6   close_datetime                3978 non-null   int64  
 7   quote_asset_volume            3978 non-null   float64
 8   number_of_trades              3978 non-null   int64  
 9   taker_buy_base_asset_volume   3978 non-null   float64
 10  taker_buy_quote_asset_volume  3978 non-null   float64
dtypes: float64(8), int64(3)
memory usage: 342.0 KB
None


Посмотрим на данные

In [29]:
df.head()


,open_datetime,open,high,low,close,volume,close_datetime,quote_asset_volume,number_of_trades,taker_buy_base_asset_volume,taker_buy_quote_asset_volume
0,1680307200000,28465.36,28481.79,28448.94,28453.55,157.35626,1680307499999,4.478911e+06,4773,63.60837,1.810491e+06
1,1680307500000,28453.55,28494.24,28451.77,28493.54,100.46294,1680307799999,2.861107e+06,3905,54.55331,1.553676e+06
2,1680307800000,28493.54,28500.00,28479.16,28500.00,71.30077,1680308099999,2.031327e+06,2946,46.54033,1.325911e+06
3,1680308100000,28499.99,28547.20,28499.99,28539.98,195.85763,1680308399999,5.588255e+06,4422,124.25112,3.545269e+06
4,1680308400000,28539.98,28539.99,28468.06,28480.01,168.33502,1680308699999,4.798562e+06,4256,79.87367,2.277018e+06


Далее возьмем по классике только цену закрытия и попробуем на ее основе делать предсказания. Я бы также добавил количество трейдов и объем. А возможно даже всё остальные данные кроме нескольких, но в рамках текущего эксперимента нам будет достаточно этих данных

In [30]:
date_column_name = 'open_datetime'
close_column_name = 'close'

df[date_column_name] = pd.to_datetime(df[date_column_name], unit='ms')
# df.set_axis(df[date_column_name], inplace=True)
df.set_index([date_column_name], inplace=True)
# df.drop(columns=[date_column_name], inplace=True)
df = df[[close_column_name]]  # .drop(columns=['Open', 'High', 'Low', 'Volume'], inplace=True)


In [31]:
orig_close = go.Scatter(
    x = df.index,
    y = df[close_column_name],
    mode = 'lines',
    name = 'Price'
)
layout = go.Layout(
    title = Set.symbol,
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Close"}
)
fig = go.Figure(data=[orig_close], layout=layout)
fig.show()


Как помним, мы не используем масштабирование

In [32]:
# def custom_scaler(data):
#     """
#     Масштабирует входные данные в диапазоне от 0 до 1.
#     """
#     min_val = np.min(data)
#     max_val = np.max(data)
#     scaled_data = (data - min_val) / (max_val - min_val)
#     return scaled_data, min_val, max_val

# def custom_unscaler(scaled_data, min_val, max_val):
#     """
#     Unscales the data that was scaled using the scaler function.
#     """
#     unscaled_data = (scaled_data * (max_val - min_val)) + min_val
#     return unscaled_data


In [33]:
# scaled_data, min_vals, max_vals = custom_scaler(df)

# # теперь можем использовать переменную scaled_data в нашей модели TensorFlow Keras, посмотрим на значения
# scaled_data.head()


Делим данные

In [34]:
close_data = df[close_column_name].values
close_data = close_data.reshape((-1,1))

split_percent = 0.80
split = int(split_percent*len(close_data))

close_train = close_data[:split]
close_test = close_data[split:]

date_train = df.index[:split]
date_test = df.index[split:]

print(len(close_train))
print(len(close_test))

3182
796


> 3. С использованием библиотеки NumPy, разделить данные на обучающую и тестовую выборки.

In [35]:
look_back = 22 # Количество значений (баров/свечей) на которых модель будет обучаться и в дальнейшем предсказывать 

train_generator = TimeseriesGenerator(close_train, close_train, length=look_back, batch_size=50)     
test_generator = TimeseriesGenerator(close_test, close_test, length=look_back, batch_size=1)


> 4. Создать и обучить LSTM-модель с использованием библиотеки Keras или TensorFlow на основе подготовленных данных. Подобрать архитектуру и гиперпараметры модели, чтобы достичь наилучшего качества прогнозирования.

In [36]:
from keras.models import Sequential
from keras.layers import LSTM, Dense

model = Sequential()
model.add(
    LSTM(50,
        activation='relu',
        input_shape=(look_back,1))
)
model.add(Dense(1))
model.compile(optimizer='adam', loss='mse')

num_epochs = 50
model.fit(train_generator, epochs=num_epochs, verbose=1)


Epoch 1/50
64/64 [==============================] - 2s 10ms/step - loss: 519651360.0000
Epoch 2/50
64/64 [==============================] - 1s 10ms/step - loss: 377316768.0000
Epoch 3/50
64/64 [==============================] - 1s 10ms/step - loss: 502752000.0000
Epoch 4/50
64/64 [==============================] - 1s 11ms/step - loss: 607863808.0000
Epoch 5/50
64/64 [==============================] - 1s 10ms/step - loss: 385239904.0000
Epoch 6/50
64/64 [==============================] - 1s 10ms/step - loss: 250854080.0000
Epoch 7/50
64/64 [==============================] - 1s 11ms/step - loss: 564502976.0000
Epoch 8/50
64/64 [==============================] - 1s 10ms/step - loss: 146625248.0000
Epoch 9/50
64/64 [==============================] - 1s 10ms/step - loss: 124593976.0000
Epoch 10/50
64/64 [==============================] - 1s 10ms/step - loss: 174431408.0000
Epoch 11/50
64/64 [==============================] - 1s 11ms/step - loss: 128227288.0000
Epoch 12/50
64/64 [===========

В этот раз не сохраняем модель

In [37]:
# model.save(f'models/{Set.symbol.lower()}_{Set.timeframe}.h5')


И приступаем к тестированию

In [38]:
prediction = model.predict(test_generator)

close_train = close_train.reshape((-1))
close_test = close_test.reshape((-1))
prediction = prediction.reshape((-1))


774/774 [==============================] - 2s 2ms/step


Посмотрим как модель вела себя во время тренировки

In [39]:
trace1 = go.Scatter(
    x = date_train,
    y = close_train,
    mode = 'lines',
    name = 'Data'
)
trace2 = go.Scatter(
    x = date_test,
    y = prediction,
    mode = 'lines',
    name = 'Prediction'
)
trace3 = go.Scatter(
    x = date_test,
    y = close_test,
    mode='lines',
    name = 'Ground Truth'
)

fig = go.Figure(data=[trace1, trace2, trace3], layout=layout)
fig.show()


> 5. Оценить качество модели на тестовой выборке, используя подходящие метрики, такие как среднеквадратичная ошибка (MSE) или средняя абсолютная ошибка (MAE).

In [40]:
mse = keras.losses.MeanSquaredError()
print("MSE:",mse(close_test[:-look_back], prediction).numpy())

mae = keras.losses.MeanAbsoluteError()
print("MAE:",mae(close_test[:-look_back], prediction).numpy())


MSE: 3421.0063
MAE: 43.014565


> 6. Визуализировать предсказания модели для тестовой выборки на графике с помощью библиотеки Matplotlib или Plotly. Вывести график с предсказанием на час вперед от последних полученных данных.

In [41]:
close_data = close_data.reshape((-1))

def predict(num_prediction, model):
    prediction_list = close_data[-look_back:]
    
    for _ in range(num_prediction):
        x = prediction_list[-look_back:]
        x = x.reshape((1, look_back, 1))
        out = model.predict(x)[0][0]
        prediction_list = np.append(prediction_list, out)
    prediction_list = prediction_list[look_back-1:]
        
    return prediction_list

def timeframe_convert(timeframe):
    time = int(re.findall("[0-9]+", timeframe)[0])
    frame = str(re.findall("[a-zA-Z]+", timeframe)[0])
    if frame == 'm':
        frame_converted = "T"
        time_converted = time
    elif frame == 'h':
        frame_converted = "T"
        time_converted = time*60
    elif converted == 'd':    
        frame_converted = "D"
        time_converted = time
    return str(time_converted) + frame_converted


def predict_dates(num_prediction):
    last_date = df.index.values[-1]
    prediction_dates = pd.date_range(last_date, periods=num_prediction+1, freq=timeframe_convert(Set.timeframe)).tolist()
    return prediction_dates

num_prediction = 12  # Предсказываем на час вмеред, т.к у нас 5ти минутный таймфрейм нам нужны следующие 12 свечей 
forecast = predict(num_prediction, model)
forecast_dates = predict_dates(num_prediction)


1/1 [==============================] - 0s 18ms/step


И теперь самое главное, посмотрим на наше предсказание

In [42]:
trace1 = go.Scatter(
    x = date_test,
    y = close_test,
    mode = 'lines',
    name = 'Ground Truth'
)

trace2 = go.Scatter(
    x = forecast_dates,
    y = forecast,
    mode = 'lines',
    name = 'Prediction'
)

layout = go.Layout(
    title = Set.symbol,
    xaxis = {'title' : "Date"},
    yaxis = {'title' : "Close"}
)


fig = go.Figure(data=[trace1, trace2], layout=layout)
fig.show()


Как видим в следующий час тренд на повышение, берем шорт под 125х Правда я бы так не рискова. Предсказания на 5 минутном таймфрейме основанные только на данных цены не стоит использоваться для реальной торговли.